In [1]:
import os
import psycopg2

from astropy import units as u
from astropy.table import Table, join, unique, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky, search_around_sky

from desitarget.targets import decode_targetid, encode_targetid, resolve
from desitarget.io import releasedict, release_to_photsys

from tqdm.notebook import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


from astropy.table import Table, vstack, hstack

import psycopg2

from tqdm.notebook import tqdm_notebook

import time


In [2]:
tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_iron_healpix.fits')
tiron

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061
2412337059332105,123.256011148025,36.2652948002806,6448,main,0.0799478243014975,3.26415764231537e-05,4,1.08863945491612,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061
2852141710442501,123.270569418049,36.2759891261316,6448,main,0.125814897161802,5.25734210595906e-05,4,4.58732237294316,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.2705694180487,36.27598912613165
2852189789749253,124.454417230594,38.7904829812355,6454,main,0.0209014343963839,8.37593315854385e-06,0,921.681470951618,iron/healpix/main/dark/64/6454/redrock-main-dark-6454.fits,EXT,1075147,124.45441723059355,38.79048298123546
2412690689490952,122.179783442643,57.7556637029528,7696,main,1.67700904770652,0.000194559932487755,0,10.886064812541,iron/healpix/main/dark/76/7696/redrock-main-dark-7696.fits,EXT,532264,122.17978344264326,57.75566370295283


In [2]:
#Remove the condition that targetid>0 because there are a few entries in the iron table that had a targetid<0.

def get_tf_targets_modified(redux, use_cached=False, verbose=False):
    """Get TF targets from the DESI observations DB for a given spectroscopic reduction.
    
    Parameters
    ----------
    redux : str
        Spectroscopic reduction. E.g., 'everest', 'fuji', ...
    use_cached : bool
        Use cached data rather than re-running the query.
    
    Returns
    -------
    
    tf_targets : Table
        Table of Tully-Fisher observations.
    """
    tf_targets = None

    if os.path.exists(f'tf_targets_{redux}.fits') and use_cached:
        tf_targets = Table.read('cache/tf_targets_{redux}.fits')
    else:
        try:
            db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password='')
            cursor = db.cursor()

            query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;"""
                         # AND pv.sga_id IS NOT NULL AND (pv.pvtype LIKE 'TFT' or pv.pvtype LIKE 'EXT' or pv.pvtype LIKE 'SGA');"""
            
            if verbose:
                print(query)

            cursor.execute(query)
            rows = cursor.fetchall()
            tf_targets = Table(list(map(list, zip(*rows))),
                               names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'Z', 'ZERR', 'SPECTYPE', 'DELTACHI2', 'ZWARN', 'PVTYPE', 'SGA_ID'])

            #- Select only targets with SGA IDs and PV types matching SGA, EXT, and TFT
            select = (tf_targets['SGA_ID'] != None) & \
                     ((tf_targets['PVTYPE'] == 'TFT') | \
                      (tf_targets['PVTYPE'] == 'EXT') | \
                      (tf_targets['PVTYPE'] == 'SGA'))
            tf_targets = tf_targets[select]

            #- Use TARGETID to extract the photometric system used during targeting
            _, _, releases, _, _, _ = decode_targetid(tf_targets['TARGETID'].value)

            photsys = []
            for i, release in enumerate(releases):
                ps = None

                if release in releasedict:
                    ps = release_to_photsys([release])[0].decode('utf-8')
                else:
                    #- Fall-through case: not all SGA center observations are in the main survey.
                    #  In this case, select 'N' or 'S' based on the SGA object's position.
                    ra  = tf_targets['TARGET_RA'][i]
                    dec = tf_targets['TARGET_DEC'][i]
                    c = SkyCoord(ra=ra, dec=dec, unit='degree')

                    #- N: in galactic northern hemisphere and with dec > 32.375. Else, S.
                    isnorth = (c.galactic.b > 0) & (dec > 32.375)
                    ps = 'N' if isnorth else 'S'

                photsys.append(ps)

            #- Complain if the photsys table doesn't match the size of the Vrot table.
            if len(photsys) != len(tf_targets):
                print(f'photsys array of len {len(photsys)} != targets array of len {len(tf_targets)}')

            tf_targets['PHOTSYS'] = photsys

            # tf_targets.write(f'cache/tf_targets_{redux}.fits', overwrite=True)

        except Exception as error:
            print(error)
        finally:
            if db is not None:
                db.close()

    return tf_targets

In [ ]:
iron = get_tf_targets_modified('iron', verbose=True)
iron

In [ ]:
#Fix the fact that for some reason the SGA_ID column was doing weird stuff and not letting me save the table, and then save the table
iron['SGA_ID'] = [str(item) for item in iron['SGA_ID']]
iron.write('test_iron2.fits', overwrite=True)

In [4]:
#Read in iron table from new query
test= Table.read('test_iron2.fits')
test

TARGETID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN,PVTYPE,SGA_ID,PHOTSYS
int64,float64,float64,float64,float64,bytes6,float64,int64,bytes3,bytes7,bytes1
-411444222,156.2181500220905,7.1584828480845655,0.1024816376852125,9.091035224678232e-05,GALAXY,0.046789859188720584,2053,TFT,4614,S
-256075243,267.7127650902238,29.4851124724098,0.024937613795555466,2.443082948450574e-05,GALAXY,1544.5199036816193,2560,SGA,447725,S
-255474428,318.1164926597467,-5.284018993812088,0.04895363299977777,7.857217657641379e-05,GALAXY,67.71628514182521,512,TFT,342501,S
-239472210,323.5029072575962,2.5962361580108633,0.05705457943374285,2.4737644271185894e-05,GALAXY,36.335152571089566,2560,TFT,1245160,S
-236269010,159.44155130148636,11.81644015613326,0.053693794815231316,2.0054150033577103e-06,GALAXY,17253.95412647724,2560,SGA,112426,S
-220511122,158.60935975565818,4.101297858301211,0.14424105964001674,2.3386872353021118e-05,GALAXY,100.71361944824457,2560,TFT,1285714,S
-215282046,135.38170916974107,2.899429822113334,0.0706704079002097,9.790452524988413e-06,GALAXY,552.1510838589165,2560,TFT,730406,S
-215264164,139.65430579472775,0.3272206295408149,0.16939400371460273,1.6665502506260876e-05,GALAXY,1103.0728027746081,2560,SGA,460051,S
-211637396,214.40836786160398,38.16543192183721,0.09247420765298109,8.226175105351364e-06,GALAXY,587.4189228154719,2560,TFT,908460,N


In [5]:
#Compare the tables from the two different queries

#Calculate the number of unique target ids that appear in each table

unique_values_set = set(tiron['TARGETID'])

print('Total values for the original query=', len(tiron['TARGETID']))
print('unique values for the original=',len(unique_values_set))


unique_in_test = set(test['TARGETID'])

print('Total values for the test =', len(test['TARGETID']))
print('unique values for the test =',len(unique_in_test))

test_id = set(test['TARGETID'])
tiron_id = set(tiron['TARGETID'])

#Find the number of entries that are shared between the two sets
common_ids = test_id.intersection(tiron_id)
common_ids_list = list(common_ids)
print('Number of data entries the lists have in common=' ,len(common_ids_list))

#Find how many elements are not in both sets
difference_ids1 = test_id.difference(tiron_id)
different_ids1 = list(difference_ids1)

difference_ids2 = tiron_id.difference(test_id)
different_ids2 = list(difference_ids2)
print('Number of data entries in the new test not in the original query=', len(different_ids1))
print('Number of data entries in the original query not in the new test=', len(different_ids2))

Total values for the original query= 99102
unique values for the original= 96661
Total values for the test = 104074
unique values for the test = 96661
Number of data entries the lists have in common= 96661
Number of data entries in the new test not in the original query= 0
Number of data entries in the original query not in the new test= 0


In [3]:
jura = get_tf_targets_modified('jura', verbose=True)
jura

SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM jura.healpix_fibermap as rdx, static.pv as pv, jura.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;


TARGETID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN,PVTYPE,SGA_ID,PHOTSYS
int64,float64,float64,float64,float64,str6,float64,int64,str3,object,str1
-430502046,134.00013295228783,5.934552839997555,1.5211813094599733,9.536633276548175e-05,GALAXY,21.043927863240242,2049,TFT,838970,S
-427872363,61.981718642816155,-22.823913110732356,0.049085960824979194,8.765385426009584e-06,GALAXY,320.327581204474,2049,TFT,982213,S
-411444222,156.2181500220905,7.1584828480845655,0.7655265334224575,0.00014819907104570706,GALAXY,2.862433671951294,2053,TFT,4614,S
-261707523,51.18809619857989,-15.380001206793725,0.11706841864860068,1.263477560383397e-05,GALAXY,5022.345633155666,512,SGA,788458,S
-260779407,138.32747528464944,17.5639019859795,0.08569271543378945,9.438671035622181e-06,GALAXY,243.51359286904335,2560,TFT,735997,S
-256075243,267.7127650902238,29.4851124724098,0.024938748634094107,2.4256075629217488e-05,GALAXY,1541.527919426706,2560,SGA,447725,S
-255779143,324.7514856412927,-14.285073192391051,0.06222014597085957,1.2531392899106468e-05,GALAXY,13019.724023681134,2560,SGA,344696,S
-255474428,318.1164926597467,-5.284018993812088,0.048956092264502225,7.806082205296064e-05,GALAXY,65.2918488328578,512,TFT,342501,S
-253178294,171.33824183216043,53.64134341804524,0.033601461542572625,3.5901373601923224e-06,GALAXY,2593.072873726487,2560,TFT,32805,N


In [4]:
#Save the JURA test table and see what it looks like

jura['SGA_ID'] = [str(item) for item in jura['SGA_ID']]
jura.write('jura_test.fits', overwrite=True)
jura_test= Table.read('jura_test.fits')
jura_test

TARGETID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN,PVTYPE,SGA_ID,PHOTSYS
int64,float64,float64,float64,float64,bytes6,float64,int64,bytes3,bytes7,bytes1
-430502046,134.00013295228783,5.934552839997555,1.5211813094599733,9.536633276548175e-05,GALAXY,21.043927863240242,2049,TFT,838970,S
-427872363,61.981718642816155,-22.823913110732356,0.049085960824979194,8.765385426009584e-06,GALAXY,320.327581204474,2049,TFT,982213,S
-411444222,156.2181500220905,7.1584828480845655,0.7655265334224575,0.00014819907104570706,GALAXY,2.862433671951294,2053,TFT,4614,S
-261707523,51.18809619857989,-15.380001206793725,0.11706841864860068,1.263477560383397e-05,GALAXY,5022.345633155666,512,SGA,788458,S
-260779407,138.32747528464944,17.5639019859795,0.08569271543378945,9.438671035622181e-06,GALAXY,243.51359286904335,2560,TFT,735997,S
-256075243,267.7127650902238,29.4851124724098,0.024938748634094107,2.4256075629217488e-05,GALAXY,1541.527919426706,2560,SGA,447725,S
-255779143,324.7514856412927,-14.285073192391051,0.06222014597085957,1.2531392899106468e-05,GALAXY,13019.724023681134,2560,SGA,344696,S
-255474428,318.1164926597467,-5.284018993812088,0.048956092264502225,7.806082205296064e-05,GALAXY,65.2918488328578,512,TFT,342501,S
-253178294,171.33824183216043,53.64134341804524,0.033601461542572625,3.5901373601923224e-06,GALAXY,2593.072873726487,2560,TFT,32805,N


In [5]:
print('Total values for the original query=', len(jura_test['TARGETID']))
print('unique values for the original=',len(set(jura_test['TARGETID'])))

Total values for the original query= 235167
unique values for the original= 230250


In [9]:
print(len(np.unique(jura_test['PVTYPE'])), np.unique(jura_test['PVTYPE'], return_counts=True))

3 (<Column name='PVTYPE' dtype='bytes3' length=3>
EXT
SGA
TFT, array([  3717,  65760, 165690]))


In [18]:
#Try the new query on the everest table to see if it recovers everything like Iron

everest = get_tf_targets_modified('everest', verbose=True)

#Fix the fact that for some reason the SGA_ID column was doing weird stuff and not letting me save the table, and then save the table
everest['SGA_ID'] = [str(item) for item in everest['SGA_ID']]
everest.write('test_everest.fits', overwrite=True)
everest_test= Table.read('test_everest.fits')
everest_test


SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM everest.healpix_fibermap as rdx, static.pv as pv, everest.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;


TARGETID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN,PVTYPE,SGA_ID,PHOTSYS
int64,float64,float64,float64,float64,bytes6,float64,int64,bytes3,bytes7,bytes1
-106245656,214.40836786160398,38.16543192183721,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,3618,TFT,908460,N
-103279575,193.87126208164,-7.797971929442544,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,EXT,1232665,S
-102479394,256.2862264156173,7.990707719122054,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,3618,SGA,1127864,S
-11020318,216.24332255936653,35.35082958045987,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,TFT,1287197,N
-10865656,214.40835441201924,38.165409435947396,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,3618,TFT,908460,N
-7899575,193.8713209296783,-7.797987680218857,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,EXT,1232665,S
-7099394,256.28620837882397,7.9906791127629555,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,SGA,1127864,S
-1916815,253.1225744031108,33.719069981503495,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,TFT,276273,N
-1326276,220.05055046471682,-0.27826736320203,-0.0019956912923479522,4.1311493573349107e-48,STAR,1.942668892225729e+84,1570,TFT,328560,S


In [10]:
everest_table = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_everest_healpix.fits')
everest_table

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes1,bytes3,int64,float64,float64
2399124498415623,214.013122385853,35.3145314097487,10511,main,0.888985760833643,0.00012204664140238,4,3.5203658118844,--,EXT,1229423,214.01312238585328,35.3145314097487
2412318637948931,214.04164239659,35.3585395889941,10511,main,0.0136311722029368,3.14431330920849e-05,0,113.593752369361,--,EXT,1229423,214.04164239658957,35.35853958899406
1092899764699136,216.00143660376,34.881062410803,10515,sv3,0.511231863324341,0.000149338383958002,4,0.700528137385845,--,EXT,682254,216.0014366037598,34.881062410803
1079700659109889,216.061747702864,34.8365760573346,10515,sv3,0.449087430966155,6.39900937889053e-05,0,9.43113215267658,--,EXT,682254,216.06174770286376,34.836576057334575
1092894798643201,216.061747702864,34.8365760573346,10515,sv3,0.907325853862498,0.000152981500789795,4,0.251146875321865,--,EXT,682254,216.06174770286376,34.836576057334575
2412333884243973,249.490218609158,36.058332514444,9433,main,0.447663677679541,8.88242926270696e-05,4,2.51997595652938,--,EXT,306417,249.4902186091578,36.058332514444025
2412428541296646,246.94530009636,40.9339918585217,9471,main,1.59664910536944,0.000142278099548223,0,16.4966533521656,--,EXT,1141359,246.94530009636003,40.93399185852167
1079037296377859,208.975477166485,5.30336633406908,26091,sv3,1.40202297072998,7.13955539614258e-05,4,8.70454740524292,--,EXT,865490,208.9754771664854,5.303366334069084
1092231435911171,208.975477166485,5.30336633406908,26091,sv3,1.63417441622298,0.000157425962002011,0,10.7714080065489,--,EXT,865490,208.9754771664854,5.303366334069084


In [12]:
#Compare the tables from the two different queries

#Calculate the number of unique target ids that appear in each table

unique_values_set = set(everest_table['TARGETID'])

print('Total values for the original query=', len(everest_table['TARGETID']))
print('unique values for the original=',len(unique_values_set))


unique_in_test = set(everest_test['TARGETID'])

print('Total values for the test =', len(everest_test['TARGETID']))
print('unique values for the test =',len(unique_in_test))

everest_test_id = set(everest_test['TARGETID'])
everest_table_id = set(everest_table['TARGETID'])

#Find the number of entries that are shared between the two sets
common_ids = everest_test_id.intersection(everest_table_id)
common_ids_list = list(common_ids)
print('Number of data entries the lists have in common=' ,len(common_ids_list))

#Find how many elements are not in both sets
difference_ids1 = everest_test_id.difference(everest_table_id)
different_ids1 = list(difference_ids1)

difference_ids2 = everest_table_id.difference(everest_test_id)
different_ids2 = list(difference_ids2)
print('Number of data entries in the new test not in the original query=', len(different_ids1))
print('Number of data entries in the original query not in the new test=', len(different_ids2))

Total values for the original query= 18180
unique values for the original= 17653
Total values for the test = 19246
unique values for the test = 17653
Number of data entries the lists have in common= 17653
Number of data entries in the new test not in the original query= 0
Number of data entries in the original query not in the new test= 0


In [19]:
#Try the new query on the fuji table to see if it recovers everything like Iron

himalayas = get_tf_targets_modified('himalayas', verbose=True)

#Fix the fact that for some reason the SGA_ID column was doing weird stuff and not letting me save the table, and then save the table
himalayas.write('test_himalayas.fits', overwrite=True)
himalayas_test= Table.read('test_himalayas.fits')
himalayas_test


SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM himalayas.healpix_fibermap as rdx, static.pv as pv, himalayas.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;
Arguments "names" and "dtype" must match number of columns


AttributeError: 'NoneType' object has no attribute 'write'

In [16]:
guadalupe_table = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_guadalupe_healpix.fits')
guadalupe_table

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64
2399124498415623,214.013122385853,35.3145314097487,10511,main,1.377580664732,9.84449640590381e-05,4,5.34532008320093,guadalupe/healpix/main/bright/105/10511/redrock-main-bright-10511.fits,EXT,1229423,214.01312238585328,35.3145314097487
2412318637948931,214.04164239659,35.3585395889941,10511,main,0.0136149298283923,3.84349317724892e-05,0,105.808545001011,guadalupe/healpix/main/dark/105/10511/redrock-main-dark-10511.fits,EXT,1229423,214.04164239658957,35.35853958899406
2412333884243973,249.490218609158,36.058332514444,9433,main,0.790966031999166,5.68528719021782e-05,4,6.58435443043709,guadalupe/healpix/main/dark/94/9433/redrock-main-dark-9433.fits,EXT,306417,249.4902186091578,36.058332514444025
2412428541296646,246.94530009636,40.9339918585217,9471,main,1.59672805899643,0.00021512202669222,0,21.9079817682505,guadalupe/healpix/main/dark/94/9471/redrock-main-dark-9471.fits,EXT,1141359,246.94530009636003,40.93399185852167
2411343818784770,207.304942983762,-7.21204274377769,25721,main,0.0251758506722491,5.63666872037601e-05,0,18.4876595666283,guadalupe/healpix/main/dark/257/25721/redrock-main-dark-25721.fits,EXT,528682,207.30494298376212,-7.212042743777691
2398191546793985,196.070630789244,-5.50346819670039,25790,main,1.47619325616168,9.56944906413878e-05,4,2.18669384717941,guadalupe/healpix/main/bright/257/25790/redrock-main-bright-25790.fits,EXT,131706,196.07063078924412,-5.503468196700393
2398113503379461,192.273729147642,-8.64496061055452,26113,main,0.0052033014260005,1.4751066688564e-05,0,250.45065983828,guadalupe/healpix/main/bright/261/26113/redrock-main-bright-26113.fits,EXT,288376,192.27372914764155,-8.644960610554522
2412060990242825,267.974068380698,23.0836119738323,32545,main,1.28091277853885,9.66284176278662e-05,4,3.30605861544609,guadalupe/healpix/main/dark/325/32545/redrock-main-dark-32545.fits,EXT,600287,267.9740683806976,23.08361197383228
2411567266136067,226.331158541844,2.06661086575034,8196,main,0.36889259266065,5.63732348350446e-05,4,4.60599585622549,guadalupe/healpix/main/dark/81/8196/redrock-main-dark-8196.fits,EXT,330560,226.33115854184445,2.0666108657503446


In [ ]:
#Compare the tables from the two different queries

#Calculate the number of unique target ids that appear in each table

unique_values_set = set(everest_table['TARGETID'])

print('Total values for the original query=', len(everest_table['TARGETID']))
print('unique values for the original=',len(unique_values_set))


unique_in_test = set(everest_test['TARGETID'])

print('Total values for the test =', len(everest_test['TARGETID']))
print('unique values for the test =',len(unique_in_test))

everest_test_id = set(everest_test['TARGETID'])
everest_table_id = set(everest_table['TARGETID'])

#Find the number of entries that are shared between the two sets
common_ids = everest_test_id.intersection(everest_table_id)
common_ids_list = list(common_ids)
print('Number of data entries the lists have in common=' ,len(common_ids_list))

#Find how many elements are not in both sets
difference_ids1 = everest_test_id.difference(everest_table_id)
different_ids1 = list(difference_ids1)

difference_ids2 = everest_table_id.difference(everest_test_id)
different_ids2 = list(difference_ids2)
print('Number of data entries in the new test not in the original query=', len(different_ids1))
print('Number of data entries in the original query not in the new test=', len(different_ids2))